In [ ]:
import requests
url = 'https://raw.githubusercontent.com/Lambda-School-Labs/tagger-ds/master/notebooks/merged_data_for_KNN_with_body.csv'
myfile = requests.get(url)
open('email.csv', 'wb').write(myfile.content)

29340319

In [ ]:
import pandas as pd

labels = pd.read_csv("email.csv").drop(columns = ['email_body','subject', 'content_type', 'to_who', 'from_who', 'dates',
                      'negative', 'positive', 'compound', 'neutral',
                      'negative.1', 'neutral.1', 'positive.1', 'compound.1',
                      'final_pred.1', 'message_id', 'user_tags', 'thread_id' 
                      ])

In [ ]:
labels

,final_pred,smart_tags
0,3,['forecast']
1,5,"['travel', 'business', 'meeting', 'take', 'fun..."
2,4,"['test', 'successful', 'way']"
3,3,"['randy', 'send', 'schedule', 'salary', 'level..."
4,2,"['let', 'shoot', 'tuesday']"
...,...,...
9995,5,"['s', 'pretty', 'sexy', 'huh', 'get', 'talk', ..."
9996,2,"['copy', 'idea', 'screw']"
9997,4,"[' ', 'forward', 'eric', 'basshouect', 'troy',..."
9998,3,"['buy', 'enron', 's']"


In [ ]:
labeled_list = labels['smart_tags'].to_list()

In [ ]:
import ast
labeled_list = [inner for item in labeled_list for inner in ast.literal_eval(item)] 

In [ ]:
len(labeled_list)

1263664

In [ ]:
labeled_list = set(labeled_list)

In [ ]:
len(labeled_list)

45828

In [ ]:
list(labeled_list)

['upheld',
 'steal',
 'mciell',
 'shudder',
 'contangominimal',
 'entrance',
 'mgas',
 'fridays',
 'oreg',
 'pgdaniel',
 'illinois',
 'tenmin',
 'httptmcomsbctcgisid',
 'sli',
 'trader',
 'ligthouse',
 'monitoringc',
 'oregonbased',
 'everyonef',
 'lackey',
 'shippers',
 'relinke',
 'crl',
 'resiliency',
 'rubber',
 'kihouectect',
 'firsthalf',
 'interstate',
 'torrescorpenronenronenron',
 'waterdesalinization',
 'slr',
 'satire',
 'pardon',
 'gillispie',
 'kool',
 'tosco',
 'maxine',
 'intcnasdaq',
 'threeboat',
 'credityou',
 'sloane',
 'moise',
 'ility',
 'imagethe',
 'schema',
 'piedmonte',
 'structurally',
 'pachecocorpenronenron',
 'fant',
 'techheavy',
 'josebestardenroncom',
 'conewayenronenronxga',
 'kellilittleenroncom',
 'indicesimage',
 'mcma',
 'cauliflower',
 'holstd',
 'broka',
 'bimini',
 'compensation',
 'angelawilsonenroncom',
 'oe',
 'contrib',
 'httpenewsbuycomcgibinfloyeetmdsiblubicad',
 'cuti',
 'nteer',
 'yorknewark',
 'impliment',
 'avoi',
 'npc',
 'manx',
 'bn'

##Randon Forest Classifier

In [ ]:
# Feature / target split
X = labels['smart_tags']
y = labels['final_pred']

In [ ]:
# Do a train / test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
X_train.shape,y_train.shape, X_test.shape,y_test.shape

((7500,), (7500,), (2500,), (2500,))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import spacy

nlp = spacy.load('en_core_web_sm')

# Vectorize
def embed(docs):
    return [nlp(doc).vector for doc in docs]

# model
classifier = RandomForestClassifier(n_estimators=100, random_state=107,
                                   max_depth=7)
# Fit the model on training data
classifier.fit(embed(X_train), y_train)

# Get predictions
pred = classifier.predict(embed(X_test))

In [ ]:
classifier.score(embed(X_test), y_test)

0.7476

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

In [ ]:
from sklearn.decomposition import TruncatedSVD

# Singular value decomposition
svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)


# LSI: Latent semantic indexing
lsi = Pipeline([('vect', vect), ('svd', svd)])

In [ ]:
from sklearn.model_selection import GridSearchCV
# Pipeline
pipe = Pipeline([
                 #Vectorrizer
                 ('vect', vect),
                 #Classifier
                 ('clf', rfc)
                ])

In [ ]:
 
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  5.2min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [ ]:
accuracy = grid_search.best_score_

In [ ]:
accuracy

0.8330666666666667

In [ ]:
searches = ['trip to Austin', 'when is the first quarter report due', 
            'documentation for project', 'resolved issue with phillip', 
            'where are you on san marcos', 'vacation ideas']